# Creating a Placement Algorithm

This tutorial demonstrates how we can create a simple placement algorithm on EdgeSimPy.

Let's start by importing the EdgeSimPy modules:

In [ ]:
# First, attempt to install required dependencies
try:
    # Attempt to import EdgeSimPy and other libraries
    import edge_sim_py
    import networkx as nx
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
except ModuleNotFoundError:
    # If imports fail, install the required packages
    !pip install -q git+https://github.com/EdgeSimPy/EdgeSimPy.git@v1.1.0
    !pip install -q networkx matplotlib pandas numpy

    # Verify installations
    import edge_sim_py
    import networkx as nx
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np

# Standard imports
import random
from typing import List, Dict

# Import EdgeSimPy components
from edge_sim_py import *
from edgesimpy.components.topology import Topology
from edgesimpy.components.scheduler import Scheduler
from edgesimpy.components.application import Application
from edgesimpy.components.task import Task
from edgesimpy.core.simulation import Simulation

class CustomTopology(Topology):
    """
    Custom topology creation for the edge computing simulation
    """
    def create_topology(self):
        """
        Create a sample network topology
        """
        # Create a random graph
        graph = nx.barabasi_albert_graph(n=10, m=2)

        # Add nodes (edge devices and servers)
        for node in graph.nodes():
            # Randomly assign node types
            if random.random() < 0.3:
                # Create a server node
                server = Server(
                    identifier=node,
                    model="generic_server",
                    ram_capacity=32,  # GB
                    storage_capacity=1000,  # GB
                    processing_units=8  # CPU cores
                )
                self.add_node(server)
            else:
                # Create an edge device
                device = Device(
                    identifier=node,
                    model="smartphone",
                    ram_capacity=4,  # GB
                    storage_capacity=128,  # GB
                    processing_units=4  # CPU cores
                )
                self.add_node(device)

        # Add edges (connections between nodes)
        for edge in graph.edges():
            self.add_edge(edge[0], edge[1])

class CustomScheduler(Scheduler):
    """
    Custom task scheduling strategy
    """
    def schedule(self, task: Task):
        """
        Simple scheduling strategy
        """
        # Find the least loaded server
        servers = [node for node in self.topology.nodes if isinstance(node, Server)]
        if not servers:
            return None

        # Select server with least current load
        best_server = min(
            servers,
            key=lambda s: len(s.running_tasks) if hasattr(s, 'running_tasks') else 0
        )
        return best_server

def create_sample_tasks(num_tasks: int = 10) -> List[Task]:
    """
    Generate sample tasks for simulation
    """
    tasks = []
    for i in range(num_tasks):
        task = Task(
            identifier=f"task_{i}",
            type="computation",
            workflow=None,
            input_data=random.uniform(0.1, 10),  # MB
            output_data=random.uniform(0.1, 10),  # MB
            processing_length=random.uniform(100, 1000),  # Million Instructions
            deadline=random.uniform(10, 100)  # Time units
        )
        tasks.append(task)
    return tasks

def run_simulation():
    """
    Main simulation runner
    """
    # Create simulation components
    topology = CustomTopology()
    scheduler = CustomScheduler(topology)

    # Create tasks
    tasks = create_sample_tasks()

    # Create application
    application = Application(
        identifier="sample_app",
        tasks=tasks
    )

    # Setup and run simulation
    simulation = Simulation(
        topology=topology,
        scheduler=scheduler,
        applications=[application]
    )

    # Run the simulation
    simulation.run()

    # Optional: Collect and print results
    print("Simulation Completed!")
    print(f"Total Nodes: {len(topology.nodes)}")
    print(f"Total Tasks: {len(tasks)}")

    # Visualization can be added here
    visualize_topology(topology)

def visualize_topology(topology):
    """
    Visualize the network topology
    """
    # Create a graph for visualization
    G = nx.Graph()

    # Add nodes with color coding
    node_colors = []
    for node in topology.nodes:
        G.add_node(node.identifier)
        # Color servers blue, devices green
        node_colors.append('blue' if isinstance(node, Server) else 'green')

    # Add edges
    for u, v in topology.edges():
        G.add_edge(u, v)

    # Draw the graph
    plt.figure(figsize=(10, 6))
    nx.draw(
        G,
        with_labels=True,
        node_color=node_colors,
        node_size=300,
        font_size=8
    )
    plt.title("Edge Computing Network Topology")
    plt.show()

def main():
    """
    Main entry point of the script
    """
    try:
        run_simulation()
    except Exception as e:
        print(f"An error occurred during simulation: {e}")

if __name__ == "__main__":
    main()

# Diagnostic print to verify module installations
print("All required modules imported successfully!")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.7 MB/s eta 0:00:00


## Implementing the Placement Algorithm

In this example, we are going to create a simple placement algorithm that works according to the well-known First-Fit heuristic. In a nutshell, our algorithm will provision each service to the first edge server with available resources to host them.

In [ ]:
def my_algorithm(parameters):
    # We can always call the 'all()' method to get a list with all created instances of a given class
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if service.server == None and not service.being_provisioned:

            # Let's iterate over the list of edge servers to find a suitable host for our service
            for edge_server in EdgeServer.all():

                # We must check if the edge server has enough resources to host the service
                if edge_server.has_capacity_to_host(service=service):

                    # Start provisioning the service in the edge server
                    service.provision(target_server=edge_server)

                    # After start migrating the service we can move on to the next service
                    break

## Running the Simulation

As we're creating a placement algorithm, we must instruct EdgeSimPy that it needs to continue the simulation until all services are provisioned within the infrastructure.

To do so, let's create a simple function that will be used as the simulation's stopping criterion. EdgeSimPy will run that function at the end of each time step, halting the simulation as soon as it returns `True`.

In [ ]:
def stopping_criterion(model: object):
    # Defining a variable that will help us to count the number of services successfully provisioned within the infrastructure
    provisioned_services = 0

    # Iterating over the list of services to count the number of services provisioned within the infrastructure
    for service in Service.all():

        # Initially, services are not hosted by any server (i.e., their "server" attribute is None).
        # Once that value changes, we know that it has been successfully provisioned inside an edge server.
        if service.server != None:
            provisioned_services += 1

    # As EdgeSimPy will halt the simulation whenever this function returns True, its output will be a boolean expression
    # that checks if the number of provisioned services equals to the number of services spawned in our simulation
    return provisioned_services == Service.count()

Once we have our stopping criterion, we can finally run our simulation by creating an instance of the `Simulator` class, loading a dataset, and calling the `run_model()` method.

In [ ]:

# Creating a Simulator object
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

# Executing the simulation
simulator.run_model()

# Checking the placement output
for service in Service.all():
    print(f"{service}. Host: {service.server}")